# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: build: File exists
-- The C compiler identification is AppleClang 10.0.0.10001145
-- The CXX compiler identification is AppleClang 10.0.0.10001145
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/Users/vpomsztein/Documents/PrivateRepositories/SentimentAnalysisMetNum/notebooks
Python 3.6.5 :: Anaconda, Inc.


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

x ./._imdb_small.csv
x imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,By far this has to be one of the worst movies ...,neg,4182_10.txt
freq,6275,2,6322,2


In [5]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

#descomentar esto si se quiere tener un dataset más chico que los 6.000 totales
#text_train = text_train[:100]
#label_train = label_train[:100]

#text_test = text_test[:100]
#label_test = label_test[:100]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
from __future__ import division
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))


Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.85, min_df=0.01, max_features=50000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values


In [8]:
import sentiment
import numpy as np
import matplotlib.pyplot as plt

In [9]:
import calendar;
import time;

def f1score(prec, rec):
    f1 = 2*(prec*rec)/(prec+rec)
    return f1

def getMetrics(predictions, actualValues):
    tp = np.sum( np.logical_and(predictions == 1, actualValues == 1) )
    tn = np.sum( np.logical_and(predictions == 0, actualValues == 0) )
    fp = np.sum( np.logical_and(predictions == 1, actualValues == 0) )
    fn = np.sum( np.logical_and(predictions == 0, actualValues == 1) )

    acc = (tp + tn) / (tp+tn+fp+fn)
    if (tp + fp) == 0:
        #esto es que nunca predije que algo era positivo, o sea que nunca le pifié
        prec = 1
    else:
        prec = tp / (tp + fp)
    if (tp + fn) == 0:
        #esto es que no había positivos reales, o sea que "los agarré a todos"
        rec = 1
    else:
        rec = tp / (tp + fn)
    return acc, prec, rec

def saveData(filename, data):
    ts = calendar.timegm(time.gmtime())
    np.savetxt("{}_{}.csv".format(ts, filename), data, delimiter=",")
    
print("Done! You're a good boy 🐶")

Done! You're a good boy 🐶


In [ ]:
%%time

# Esta celda puede ser ejecutada sólo cuando se quiere probar PCA
pca = sentiment.PCA(50)
pca.fit(X_train)

In [ ]:

# Idem anterior!

pca_x_train = pca.transform(X_train)
pca_x_test = pca.transform(X_test)

In [ ]:
%%time

# Descomentar la implementación deseada:
# Uncomment this for KNN only (PCA disabled):
clf = sentiment.KNNClassifier(100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test[:500])
acc = accuracy_score(y_test[:500], y_pred[:500])

# Uncomment this for PCA enabled:
#clf = sentiment.KNNClassifier(550)
#clf.fit(pca_x_train, y_train)

#y_pred = clf.predict(pca_x_test)

#acc, prec, rec = getMetrics(y_pred, y_test)
#f1 = f1score(prec, rec)

print(acc, prec, rec, f1)

In [10]:
## ==================== Experimento 1 ========================
## Análisis de calidad de resultados en base a k y alpha

## Analizo la variación de las métricas en funcion de k

K_vals = np.arange(0,1000,50)
K_vals[0] = 1
mets = np.zeros((len(K_vals), 3))
               
for i,k in enumerate(K_vals):
#     pca = sentiment.PCA(a)
#     pca.fit(X_train[:1000])
#     Xtc_train = pca.transform(X_train[:1000])
#     Xtc_test = pca.transform(X_test[:1000])
    
    clf = sentiment.KNNClassifier(25)
    clf.fit(Xtc_train, y_train[:1000])
    preds = clf.predict(Xtc_test)
    acc, prec, rec = getMetrics(preds[:1000], y_test[:1000])
    f1 = f1score(prec, rec)
    mets[i] = [acc, prec, rec, f1]
    print("Finished {}".format(a))
    
plt.plot(K_vals,mets[:,0], 'b.-')
plt.plot(K_vals,mets[:,1], 'r.-')
plt.plot(K_vals,mets[:,2], 'g.-')
plt.plot(K_vals,mets[:,3], 'k.-')
plt.title("Metricas en funcion de Alpha")
plt.xlabel("K")
plt.ylabel("%")
plt.axis([1, K_vals[-1], 0.4, 1])
plt.gca().legend(('Acc','Prec','Recall','F1'))

In [ ]:
## Analizo la variación de las métricas en funcion de alpha para K fijo

alphas = np.arange(0,500,50)
alphas[0] = 1
K_vals_fixes = np.array([50,550,1100])

dmets = { i : np.zeros((len(alphas), 3)) for i in K_vals_fixes}
dmets

In [ ]:
print("Beginning alpha metric calculations")

for i,a in enumerate(alphas):
    
    pca = sentiment.PCA(a)
    pca.fit(X_train)
    Xtc_train = pca.transform(X_train)
    Xtc_test = pca.transform(X_test)
    print("Finished training PCA {}".format(a))

    for j, k in enumerate(K_vals_fixes):
        clf = sentiment.KNNClassifier(k)
        clf.fit(Xtc_train, y_train)
        preds = clf.predict(Xtc_test)
        acc, prec, rec = getMetrics(preds, y_test)
        f1 = f1score(prec,rec)
        mets = dmets[k]
        mets[i] = [acc, prec, rec, f1]
        print("Finished {}, alpha = {}".format(k, a))


## Metricas para PCA:

line_styles = ['b.-', 'r.-', 'g.-', 'k.-', 'y.-', 'm.-']
for j,k in enumerate(K_vals_fixes):
    mets = dmets[k]

    plt.subplot(4, 1, 1)
    plt.plot(alphas,mets[:,0], line_styles[j])
    plt.ylabel("Acc")
    plt.xticks([])

    plt.subplot(4, 1, 2)
    plt.plot(alphas,mets[:,1], line_styles[j])
    plt.ylabel("Prec")
    plt.xticks([])
    
    plt.subplot(4, 1, 3)
    plt.plot(alphas,mets[:,2], line_styles[j])
    plt.ylabel("Rec")
    plt.xticks([])
    
    plt.subplot(4, 1, 4)
    plt.plot(alphas,mets[:,2], line_styles[j])
    plt.ylabel("F1")
    plt.xticks([])

    
plt.xlabel("alpha")
plt.xticks(alphas)
plt.axis([1, alphas[-1], 0.3, 0.8])
plt.gca().legend([str(i) for i in K_vals_fixes])

In [ ]:
a = np.random.random((1200, 500))

# generate 2 2d grids for the x & y bounds
y, x = np.meshgrid(np.linspace(-1, 1200), np.linspace(1, 500))

z = (1 - x / 2. + x ** 5 + y ** 3) * np.exp(-x ** 2 - y ** 2)
# x and y are bounds, so z should be the value *inside* those bounds.
# Therefore, remove the last value from the z array.
z_min, z_max = -np.abs(z).max(), np.abs(z).max()

fig, ax = plt.subplots()

c = ax.pcolormesh(x, y, z, cmap='RdBu', vmin=z_min, vmax=z_max)
ax.set_title('pcolormesh')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
fig.colorbar(c, ax=ax)

# plt.show()

# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()
dmets
y

In [ ]:
for i in range(15, 0, -1): # <--Delete me
    print(i)
    pca = sentiment.PCA(min(i, 50))
    pca.fit(X_train[:i])
    Xtc_train = pca.transform(X_train[:i])
    Ytc_train = y_train[:i]
    Xtc_test = pca.transform(X_test)
    
    for k in range(min(i, 50), 0, -1): # <--Delete me
        clf = sentiment.KNNClassifier(k)

In [ ]:
trainSizeRange = range(X_train.shape[0], 0, -120)
trainSizeRangeLength = len(trainSizeRange)
knnRange = range(2500, 0, -50)
knnRangeLength = len(knnRange)

print("Will have {} trainSize instances".format(trainSizeRangeLength))
print("Will have {} knn instances".format(knnRangeLength))

accColorMap = np.zeros(shape=(trainSizeRangeLength, len(knnRange)))
precColorMap = np.zeros(shape=(trainSizeRangeLength, len(knnRange)))
recColorMap = np.zeros(shape=(trainSizeRangeLength, len(knnRange)))

for i in range(0, len(trainSizeRange)):
    trainSize = trainSizeRange[i]
    pca = sentiment.PCA(min(i, 50))
    pca.fit(X_train[:trainSize])
    Xtc_train = pca.transform(X_train[:trainSize])
    Ytc_train = y_train[:trainSize]
    Xtc_test = pca.transform(X_test)
    print("Finished training PCA train_size = {}".format(trainSize))

    for j in range(0, knnRangeLength):
        k = knnRange[j]
        if (k <= trainSize):
            clf = sentiment.KNNClassifier(k)
            clf.fit(Xtc_train, Ytc_train)
            preds = clf.predict(Xtc_test)
            acc, prec, rec = getMetrics(preds, y_test)
            f1 = f1score(prec,rec)
            accColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = acc
            precColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = prec
            recColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = rec
            f1ColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = f1
            print("Finished k = {}, train_size = {}".format(k, trainSize))
        else:
            accColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = 0
            precColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = 0
            recColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = 0
            f1ColorMap[trainSizeRangeLength - i - 1, knnRangeLength - j - 1] = 0

saveData("accColorMap", accColorMap)
saveData("precColorMap", precColorMap)
saveData("recColorMap", recColorMap)
saveData("f1ColorMap", f1ColorMap)
        
fig, ax = plt.subplots()

c = ax.pcolormesh(accColorMap, cmap='hot')
ax.set_title('Train_size vs neighbors')
# set the limits of the plot to the limits of the data
ax.axis([0, accColorMap.shape[1], 0, accColorMap.shape[0]])
fig.colorbar(c, ax=ax)

fig2, ax2 = plt.subplots()

c2 = ax2.pcolormesh(precColorMap, cmap='hot')
ax2.set_title('pcolormesh')
# set the limits of the plot to the limits of the data
ax2.axis([0, precColorMap.shape[1], 0, precColorMap.shape[0]])
fig2.colorbar(c2, ax=ax2)

fig3, ax3 = plt.subplots()

c3 = ax3.pcolormesh(recColorMap, cmap='hot')
ax3.set_title('pcolormesh')
# set the limits of the plot to the limits of the data
ax3.axis([0, recColorMap.shape[1], 0, recColorMap.shape[0]])
fig3.colorbar(c3, ax=ax3)

fig4, ax4 = plt.subplots()

c4 = ax4.pcolormesh(f1ColorMap, cmap='hot')
ax4.set_title('pcolormesh')
# set the limits of the plot to the limits of the data
ax4.axis([0, f1ColorMap.shape[1], 0, f1ColorMap.shape[0]])
fig4.colorbar(c4, ax=ax4)

plt.show()

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolormesh(accColorMap, cmap='hot', vmin=0, vmax=1)
ax.set_title('Train_size vs neighbors (accuracy)')
# set the limits of the plot to the limits of the data
ax.axis([0, accColorMap.shape[1], 0, accColorMap.shape[0]])
ax.set_xlabel("Train size (1:125)")
ax.set_ylabel("K neighbors (1:50)")
fig.colorbar(c, ax=ax)

fig2, ax2 = plt.subplots()

c2 = ax2.pcolormesh(precColorMap, cmap='hot', vmin=0, vmax=1)
ax2.set_title('Train_size vs neighbors (precision)')
# set the limits of the plot to the limits of the data
ax2.axis([0, precColorMap.shape[1], 0, precColorMap.shape[0]])
ax2.set_xlabel("Train size (1:125)")
ax2.set_ylabel("K neighbors (1:50)")
fig2.colorbar(c2, ax=ax2)

fig3, ax3 = plt.subplots()

c3 = ax3.pcolormesh(recColorMap, cmap='hot', vmin=0, vmax=1)
ax3.set_title('Train_size vs neighbors (recall)')
# set the limits of the plot to the limits of the data
ax3.axis([0, recColorMap.shape[1], 0, recColorMap.shape[0]])
ax3.set_xlabel("Train size (1:125)")
ax3.set_ylabel("K neighbors (1:50)")
fig3.colorbar(c3, ax=ax3)

fig3, ax3 = plt.subplots()

c4 = ax4.pcolormesh(f1ColorMap, cmap='hot', vmin=0, vmax=1)
ax4.set_title('Train_size vs neighbors (f1)')
# set the limits of the plot to the limits of the data
ax4.axis([0, f1ColorMap.shape[1], 0, f1ColorMap.shape[0]])
ax4.set_xlabel("Train size (1:125)")
ax4.set_ylabel("K neighbors (1:50)")
fig4.colorbar(c4, ax=ax4)

plt.show()

In [10]:
#EXPERIMENTO DE MATOOOO, ESCRIBE UN CSV PIOLA, CON VARIAS COLUMNAS PARA ANALIZAR LO QUE UNO QUIERA
#PUEDEN DEJAR LAS PRIMERAS TRES VARIABLES FIJAS SI NO QUIEREN EXPERIMENTAR CON EL COUNTVECTORIZER
#HAY QUE TOCAR UN POQUITO EL FOR MAS ANIDADO PARA CAMBIAR DE KNN A PCA, PERO SOLO ESO

from sklearn.feature_extraction.text import CountVectorizer
import csv

ts = calendar.timegm(time.gmtime())
name = "countVect_{}.csv".format(ts)

with open(name, "w") as csvFile:  # te hace el open y el close
    writer = csv.writer(csvFile)
    writer.writerow(['max_df', 'min_df', 'max_features', 'actual_words', 'acc', 'prec', 'rec', 'f1', 'K', 'alpha', 'timeKNN', 'timePCA'])
    #file.write("max_df,min_df,max_features,actual_words,acc,prec,rec,f1,K,alpha\n")
    #si es KNN sin PCA pongo alpha=0 en el csv
    
    for max_df in [0.20,0.90,0.98]:
        for min_df in [0.002, 0.005, 0.01, 0.02, 0.05, 0.1]:
                max_features=10000
                #no tiene mucho sentido tocarlo al max_features, porque ya estudié cuantas palabras quedan 
                #jugando con max y min, si quiero mas o menos palabras lo hago con max y min
                #el maximo en realidad va a ser con min0.002 y es 6500
                
                vectorizer = CountVectorizer(max_df=max_df, min_df=min_df, max_features=max_features)
                vectorizer.fit(text_train)
                X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
                X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

                #esto te deja ver cuantas palabras quedaron despues del CountVectorizer
                actual_words = X_train.shape
                print(X_test.shape)
                
                #es mas feo poner el alpha arriba que el K, pero acelera la experimentacion
                for alpha in [0, 10, 50, 200, 600]:  #si es sin PCA es alpha=0
                    #Si alpha>actual_words (o sea no hay tantas palabras como alpha) puede ser que se rompa..
                    #eso es lo que les pregunté por whatsapp, por las dudas pongo este if
                    if (alpha>actual_words):
                        break
                    
                    startPCA = time.time()
                    
                    pca_x_train = np.zeros(0) #la idea sería que esto nunca se use así, pero si no abajo me dice
                    pca_x_test = np.zeros(0) # que no las tengo declaradas
                    
                    if (not alpha==0): #si es sin PCA
                        pca = sentiment.PCA(alpha)
                        pca.fit(X_train)
                        pca_x_train = pca.transform(X_train)
                        pca_x_test = pca.transform(X_test)
                    
                    endPCA = time.time()
                    for K in [25, 250, 450, 550, 800, 1500]:
                        startKNN = time.time()
                            
                        clf = sentiment.KNNClassifier(K)
                        
                        if (not alpha==0): #si es sin PCA
                            clf.fit(X_train, y_train)
                            y_pred = clf.predict(X_test)
                        
                        if (alpha==0): #si es con PCA
                            clf.fit(pca_x_train, y_train)
                            y_pred = clf.predict(pca_x_test)
                        
                        end = time.time()
                        
                        acc, prec, rec = getMetrics(y_pred, y_test)
                        f1 = f1score(prec,rec)
                        timeKNN=(endKNN-startKNN)
                        timePCA=(endPCA-startPCA)
                        
                        writer.writerow([max_df, min_df, max_features, actual_words, acc, prec, rec,  f1, K, alpha, timeKNN, timePCA])
                        #file.write("{},{},{},{},{},{},{},{}\n".format(max_df,min_df,max_features,actual_words,acc,prec,rec,f1))

#hasta aca tengo abierto el archivo en el que escribo


#max df cambia muy poco, hay muy pocas palabras que aparecen más de 80% de los comentarios por ej
#incluso más que en el 20% de los comentarios no hay muchas, igual tiene sentido hacerlo quizas? para sacar molestos.
#Seguro no quiero un max_df bajo como 0.20 porque me saca las mejores palabras para comparar..
#Probar si poner max_df=0.1 por ej arruina PCA

#min_df=0.003 es el momento clave que max_features se acerca a 5000. O sea que hay como 4800 palabras que aparecen en 0.3% comentarios
#ya con min_df=0.01% tenemos como 43.000 palabras. Todas las palabras aparecen en al menos 0.01% de comments?? es raro eso
#Duh.. 0.01% ya es menos de 1 comentario.. Podría pensar que el máximo razonable para probar de min_df es 0.2%, que es aparecer
#en 12 comentarios aprox, que podemos pensar empieza a ser suficiente para sacar conclusiones.
#Eso ya te baja el Nº palabras a 6500, que es bastante cerca del máximo que queríamos tomar de 5000. Experimentar por ahí

#OJO, count vectorizer solo nos deja 43.000 palabras, que son todas las que hay en text_train, y al text_test lo reduce
#a esas palabras!!! o sea que ni cuenta las palabras que hay en los comentarios de test que no estaban en el train

#para los valores que venía por defecto de la catedra (0.9  0.01):
#segun esto, los comentarios de train quedan en promedio con 102 palabras, y los de test con 98, no es terrible, pero claramente
#les sacamos varias palabras a los de train... igualmente seguro eran las menos frecuentes, porque nunca aparecian en train

(6275, 6424)


TypeError: '>' not supported between instances of 'int' and 'tuple'